In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.read_csv('wall-robot-nav.csv')

In [ ]:
class KernelRegressionClassifier:
    _kernels = ['linear']
    def __init__(self, k=10, h=None, kernel='linear'):
        self.k = k
        self.h = h
        if kernel in self._kernels:
            self.kernel = kernel
        else
            self.kernel = 'linear'
    def get_distance(self, x, xi):
        # return np.sq

In [3]:
class KernelRegressionClassifier:
    self._kernels = ['linear']
    def __init__(k=10, h=None, kernel='linear'):
        self.k = k
        self.h = h
        if kernel in self._kernels:
            self.kernel = kernek
        else
            self.kernel = 'linear'
    def get_distance(x, xi):
        return np.sq

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V16,V17,V18,V19,V20,V21,V22,V23,V24,Class
0,0.438,0.498,3.625,3.645,5.000,2.918,5.000,2.351,2.332,2.643,...,0.593,0.502,0.493,0.504,0.445,0.431,0.444,0.440,0.429,4
1,0.438,0.498,3.625,3.648,5.000,2.918,5.000,2.637,2.332,2.649,...,0.592,0.502,0.493,0.504,0.449,0.431,0.444,0.443,0.429,4
2,0.438,0.498,3.625,3.629,5.000,2.918,5.000,2.637,2.334,2.643,...,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.446,0.429,4
3,0.437,0.501,3.625,3.626,5.000,2.918,5.000,2.353,2.334,2.642,...,0.593,0.502,0.493,0.504,0.449,0.431,0.444,0.444,0.429,4
4,0.438,0.498,3.626,3.629,5.000,2.918,5.000,2.640,2.334,2.639,...,0.592,0.502,0.493,0.504,0.449,0.431,0.444,0.441,0.429,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5451,0.910,5.000,3.997,2.785,2.770,2.572,2.433,1.087,1.772,1.040,...,0.660,0.648,0.657,0.686,5.000,1.045,5.000,5.000,1.562,1
5452,0.926,5.000,4.015,2.792,2.777,2.571,1.768,1.071,1.762,1.021,...,0.652,0.640,0.649,1.593,1.616,1.058,5.000,5.000,1.085,2
5453,0.937,5.000,4.034,2.799,2.784,2.571,1.754,1.053,1.752,1.002,...,0.648,0.633,0.642,0.741,5.000,1.065,5.000,5.000,1.105,2
5454,0.945,4.052,4.052,2.809,2.791,2.441,1.757,1.034,1.743,0.983,...,0.641,0.626,0.635,0.754,5.000,1.076,5.000,5.000,1.118,1


In [ ]:
class KernelRegressionClassifier:
    self._kernels = ['linear']
    def __init__(k=10, h=None, kernel='linear'):
        self.k = k
        self.h = h
        if kernel in self._kernels:
            self.kernel = kernek
        else
            self.kernel = 'linear'
    def get_distance(x, xi):
        return np.sq